## Load packages and data

In [1]:
## load packages
import os 
import pathlib
import gdown
import pyarrow
import pandas as pd
import numpy as np
import itertools
import re
import pooch
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from download_data import load_chorus, load_shormccarty

In [2]:
#load data
ideology = pd.read_csv("../data/shormccarty/state_ideology.tab",sep='\t')

chorus_folder = "../data/chorus/"
#block_as = pd.read_parquet(os.path.join(chorus_folder, "block_assignments.parquet"))
#positions = pd.read_parquet(os.path.join(chorus_folder, "positions.parquet"))
#clients = pd.read_parquet(os.path.join(chorus_folder, "clients.parquet"))
bills = pd.read_parquet(os.path.join(chorus_folder, "bills.parquet"))

In [3]:
##SHOULD USE STATE UNIFIED BILL ID TO IDENTIFY WHERE THE NAS ARE
bills[bills["year_start"].isna()]
#bills

,legiscan_bill_id,legiscan_bill,legiscan_url,status_date,status,last_action_date,last_action,title,description,year_start,year_end,prefile,sine_die,prior,session_title,state,bill_chamber,bill_number,bill_suffix,sponsor_list,description_ncsl,ncsl_summary,date_overlap_jaccard,title_overlap_jaccard,ncsl_topics,ncsl_databases,ncsl_metatopics,state_unified_bill_id
655211,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NJ,None,None,None,None,None,None,NaN,NaN,None,None,None,NJ_S_0000001235_2014
655212,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NJ,None,None,None,None,None,None,NaN,NaN,None,None,None,NJ_S_0000000017_2014
655213,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NJ,None,None,None,None,None,None,NaN,NaN,None,None,None,NJ_S_0000003631_2014
655214,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NJ,None,None,None,None,None,None,NaN,NaN,None,None,None,NJ_S_0000004000_2014
655215,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NJ,None,None,None,None,None,None,NaN,NaN,None,None,None,NJ_A_0000005330_2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707285,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_S_0000002235_2019
707286,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_H_0000007127_2019
707287,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_H_0000007200_2019
707288,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_S_0000002670_2019


In [9]:
#save only relevant years from ideology 
bill_years = np.concatenate((bills["year_start"].dropna().unique(), (bills["year_end"].dropna().unique())))
ideology = ideology[ideology["year"].isin(bill_years)]

In [11]:
#ideology rating should be matched with status_date
ideology


,st,alpha,fips,icpsr,year,hou_chamber,sen_chamber,hou_dem,hou_rep,hou_majority,hou_minority,hou_dem_mean,hou_rep_mean,sen_dem,sen_rep,sen_majority,sen_minority,sen_dem_mean,sen_rep_mean,h_diffs,s_diffs,h_distance,s_distance,h_dem_sd,s_dem_sd,h_rep_sd,s_rep_sd,h_chamber_sd,s_chamber_sd,hou_dem_error,hou_chamber_error,hou_rep_error,hou_majority_error,sen_dem_error,sen_chamber_error,sen_rep_error,sen_majority_error
14,AK,2,2,81,2007,0.523,0.468,-0.698,0.791,0.791,-0.698,-0.589128,0.719257,-0.703,0.710,0.710,-0.703,-0.856106,0.669533,1.489,1.413,0.797832,0.909986,0.408929,0.301721,0.198455,0.142198,0.717924,0.809548,0.050,0.039,0.031,0.031,0.080,0.044,0.042,0.042
15,AK,2,2,81,2008,0.507,0.468,-0.698,0.768,0.768,-0.698,-0.589128,0.713316,-0.703,0.710,0.710,-0.703,-0.856106,0.669533,1.466,1.413,0.802283,0.909986,0.408929,0.301721,0.200721,0.142198,0.718686,0.809548,0.052,0.036,0.033,0.033,0.080,0.048,0.042,0.042
16,AK,2,2,81,2009,0.480,NaN,-0.650,0.818,0.818,-0.650,-0.533628,0.731638,NaN,NaN,NaN,NaN,NaN,NaN,1.468,NaN,0.812018,NaN,0.442372,NaN,0.224663,NaN,0.720321,NaN,0.044,0.038,0.034,0.034,NaN,NaN,NaN,NaN
17,AK,2,2,81,2010,0.393,NaN,-0.650,0.781,0.781,-0.650,-0.567197,0.719462,NaN,NaN,NaN,NaN,NaN,NaN,1.431,NaN,0.808474,NaN,0.364706,NaN,0.225243,NaN,0.711011,NaN,0.043,0.036,0.034,0.034,NaN,NaN,NaN,NaN
18,AK,2,2,81,2011,0.532,-0.143,-0.650,0.755,0.755,-0.650,-0.571214,0.722788,-0.690,0.681,-0.690,0.681,-0.837136,0.672819,1.405,1.371,0.790119,0.954743,0.389079,0.294996,0.214656,0.292690,0.705385,0.825706,0.047,0.040,0.035,0.035,0.081,0.136,0.041,0.081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,WY,50,56,68,2014,0.395,0.429,-0.848,0.440,0.440,-0.848,-0.907917,0.541724,-0.988,0.482,0.482,-0.988,-0.992664,0.514360,1.288,1.470,0.697650,0.569977,0.302773,0.154534,0.410693,0.237335,0.635315,0.567908,0.057,0.021,0.022,0.022,0.067,0.027,0.028,0.028
1296,WY,50,56,68,2015,0.413,0.453,-0.934,0.493,0.493,-0.934,-0.904634,0.630172,-0.988,0.489,0.489,-0.988,-0.992664,0.503668,1.427,1.477,0.784911,0.578030,0.246426,0.154534,0.464994,0.249124,0.704914,0.568880,0.063,0.022,0.026,0.026,0.066,0.028,0.026,0.026
1297,WY,50,56,68,2016,0.413,0.453,-0.934,0.493,0.493,-0.934,-0.904634,0.631138,-0.988,0.489,0.489,-0.988,-0.992664,0.506852,1.427,1.477,0.784111,0.576902,0.246426,0.154534,0.463972,0.247248,0.704616,0.569177,0.067,0.022,0.025,0.025,0.066,0.026,0.026,0.026
1298,WY,50,56,68,2017,0.444,0.411,-0.897,0.519,0.519,-0.897,-0.884320,0.649465,-1.111,0.477,0.477,-1.111,-1.139733,0.512596,1.416,1.588,0.822834,0.560047,0.256933,0.290917,0.525663,0.265388,0.740371,0.568482,0.070,0.025,0.028,0.028,0.115,0.026,0.032,0.032


In [ ]:
#map ideology measures


In [12]:
#map passed bills
bills.set_index('state_unified_bill_id').status.isin([4, 5]).to_dict()

{'KS_HB_0000002001_2013': False,
 'KS_HB_0000002002_2013': True,
 'KS_HB_0000002011_2013': True,
 'KS_HB_0000002101_2013': True,
 'KS_SB_0000000118_2013': True,
 'KS_SB_0000000119_2013': False,
 'KS_SB_0000000120_2013': True,
 'KS_SB_0000000121_2013': False,
 'KS_SB_0000000122_2013': True,
 'KS_SB_0000000123_2013': False,
 'KS_SB_0000000124_2013': True,
 'KS_SB_0000000125_2013': False,
 'KS_SB_0000000126_2013': False,
 'KS_SB_0000000127_2013': False,
 'KS_HB_0000002102_2013': False,
 'KS_SB_0000000128_2013': True,
 'KS_SB_0000000129_2013': True,
 'KS_SB_0000000130_2013': False,
 'KS_SB_0000000131_2013': False,
 'KS_SB_0000000132_2013': False,
 'KS_SB_0000000133_2013': False,
 'KS_SB_0000000134_2013': False,
 'KS_SB_0000000135_2013': True,
 'KS_SB_0000000136_2013': True,
 'KS_SB_0000000137_2013': False,
 'KS_HB_0000002103_2013': False,
 'KS_SB_0000000138_2013': False,
 'KS_SB_0000000139_2013': True,
 'KS_SB_0000000140_2013': False,
 'KS_SB_0000000141_2013': False,
 'KS_SB_0000000142_201

In [37]:
bills_test = bills.loc[500:600]
bills

,legiscan_bill_id,legiscan_bill,legiscan_url,status_date,status,last_action_date,last_action,title,description,year_start,year_end,prefile,sine_die,prior,session_title,state,bill_chamber,bill_number,bill_suffix,sponsor_list,description_ncsl,ncsl_summary,date_overlap_jaccard,title_overlap_jaccard,ncsl_topics,ncsl_databases,ncsl_metatopics,state_unified_bill_id
29,450474.0,HB2001,https://legiscan.com/KS/bill/HB2001/2013,2014-05-30,6.0,2014-05-30,House Died in Committee,"Substitute HB 2001 by Committee on Veterans, M...","Substitute HB 2001 by Committee on Veterans, M...",2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,KS,HB,2001,None,"['Veterans, Military and Homeland Security']",National Guard Educational Assistance Act,Relates to the national guard educational assi...,0.067164,0.263158,ncsl_database__military_and_veterans_state_leg...,ncsl_database__military_and_veterans_state_leg...,ncsl_metatopic__military_and_veterans_affairs,KS_HB_0000002001_2013
30,450475.0,HB2002,https://legiscan.com/KS/bill/HB2002/2013,2014-04-30,4.0,2014-04-30,"House Approved by Governor on Thursday, 10 Apr...",Substitute for HB 2002 by Committee on Federal...,Substitute for HB 2002 by Committee on Federal...,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,KS,HB,2002,None,['Federal and State Affairs'],None,None,NaN,NaN,None,None,None,KS_HB_0000002002_2013
31,451743.0,HB2011,https://legiscan.com/KS/bill/HB2011/2013,2013-05-08,4.0,2013-05-08,"House Approved by Governor on Tuesday, 16 Apri...",Senate Substitute for HB 2011 by Committee on ...,Senate Substitute for HB 2011 by Committee on ...,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,KS,HB,2011,None,['Transportation'],None,None,NaN,NaN,None,None,None,KS_HB_0000002011_2013
32,472087.0,HB2101,https://legiscan.com/KS/bill/HB2101/2013,2014-04-30,4.0,2014-04-30,"House Approved by Governor on Wednesday, 16 Ap...",Senate Substitute for HB 2101 by Committee on ...,Senate Substitute for HB 2101 by Committee on ...,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,KS,HB,2101,None,['Utilities'],None,None,NaN,NaN,None,None,None,KS_HB_0000002101_2013
33,485433.0,SB118,https://legiscan.com/KS/bill/SB118/2013,2013-04-05,4.0,2013-04-05,"Senate Approved by Governor on Friday, 05 Apri...",Relating to law enforcement reporting and inve...,Relating to law enforcement reporting and inve...,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,KS,SB,118,None,['Greg Smith'],None,None,NaN,NaN,None,None,None,KS_SB_0000000118_2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707285,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_S_0000002235_2019
707286,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_H_0000007127_2019
707287,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_H_0000007200_2019
707288,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,RI,None,None,None,None,None,None,NaN,NaN,None,None,None,RI_S_0000002670_2019


## data: BILLS

In [ ]:
#bills[bills.duplicated("legiscan_bill_id", keep = False)].dropna(how="all")
#bills[bills.duplicated("state_unified_bill_id", keep = False)].dropna(how="all")
bills[bills["state_unified_bill_id"] == "NE_L_0000000941_2013"]

,legiscan_bill_id,legiscan_bill,legiscan_url,status_date,status,last_action_date,last_action,title,description,year_start,year_end,prefile,sine_die,prior,session_title,state,bill_chamber,bill_number,bill_suffix,sponsor_list,description_ncsl,ncsl_summary,date_overlap_jaccard,title_overlap_jaccard,ncsl_topics,ncsl_databases,ncsl_metatopics,state_unified_bill_id
8830,591946.0,LB941,https://legiscan.com/NE/bill/LB941/2013,2014-03-31,4.0,2014-03-31,"Approved by Governor on March 28, 2014",Provide for a dairy growth study,Provide for a dairy growth study,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,NE,LB,941,None,['Ken Schilz'],None,None,NaN,NaN,None,None,None,NE_L_0000000941_2013
8831,633963.0,LB941A,https://legiscan.com/NE/bill/LB941A/2013,2014-03-31,4.0,2014-03-31,"Approved by Governor on March 28, 2014",Appropriation Bill,Appropriation Bill,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,NE,LB,941,A,['Ken Schilz'],None,None,NaN,NaN,None,None,None,NE_L_0000000941_2013


,legiscan_bill_id,legiscan_bill,legiscan_url,status_date,status,last_action_date,last_action,title,description,year_start,year_end,prefile,sine_die,prior,session_title,state,bill_chamber,bill_number,bill_suffix,sponsor_list,description_ncsl,ncsl_summary,date_overlap_jaccard,title_overlap_jaccard,ncsl_topics,ncsl_databases,ncsl_metatopics,state_unified_bill_id
8830,591946.0,LB941,https://legiscan.com/NE/bill/LB941/2013,2014-03-31,4.0,2014-03-31,"Approved by Governor on March 28, 2014",Provide for a dairy growth study,Provide for a dairy growth study,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,NE,LB,941,None,['Ken Schilz'],None,None,NaN,NaN,None,None,None,NE_L_0000000941_2013
8831,633963.0,LB941A,https://legiscan.com/NE/bill/LB941A/2013,2014-03-31,4.0,2014-03-31,"Approved by Governor on March 28, 2014",Appropriation Bill,Appropriation Bill,2013.0,2014.0,0.0,1.0,1.0,2013-2014 Regular Session,NE,LB,941,A,['Ken Schilz'],None,None,NaN,NaN,None,None,None,NE_L_0000000941_2013


In [14]:
##OVERVIEW
#column labels
print(bills.columns)
#length (bills)
print(len(bills))
#length per state
print(bills.value_counts(subset=["state"]))

Index(['legiscan_bill_id', 'legiscan_bill', 'legiscan_url', 'status_date',
       'status', 'last_action_date', 'last_action', 'title', 'description',
       'year_start', 'year_end', 'prefile', 'sine_die', 'prior',
       'session_title', 'state', 'bill_chamber', 'bill_number', 'bill_suffix',
       'sponsor_list', 'description_ncsl', 'ncsl_summary',
       'date_overlap_jaccard', 'title_overlap_jaccard', 'ncsl_topics',
       'ncsl_databases', 'ncsl_metatopics', 'state_unified_bill_id'],
      dtype='object')
588786
state
TX       96555
IL       92779
NJ       67914
MA       52677
FL       36595
MD       34312
RI       30057
MO       28700
IA       23035
WI       21952
AZ       21768
NE       17523
OH       16032
CO       13757
SD       13465
MT       11417
KS       10248
Name: count, dtype: int64


In [20]:
#OVERVIEW: status
bills["status"].isin([4,5]).value_counts()

status
False    450203
True     138583
Name: count, dtype: int64

In [4]:
#identify unique combis of metatopic, topic and database
bills_topics = bills[["ncsl_metatopics","ncsl_topics"]].dropna(how = "all", ignore_index = True) #extract topics and metatopics
#split both cols on "; "
bills_topics["ncsl_topics"] = bills_topics["ncsl_topics"].str.split("; ")
bills_topics["ncsl_metatopics"] = bills_topics["ncsl_metatopics"].str.split("; ")
#explode both cols to make sure that each unique combo has a row
bills_topics = bills_topics.explode("ncsl_topics", ignore_index=True)
bills_topics = bills_topics.explode("ncsl_metatopics", ignore_index=True)
#remove 'ncsl_metatopics" from metatopic col
bills_topics["ncsl_metatopics"] = bills_topics["ncsl_metatopics"].str.replace("ncsl_metatopic__", "")
#remove duplicates
print(f"number of rows before dropping duplicates: {len(bills_topics)}") #153220
bills_topics = bills_topics.drop_duplicates(ignore_index = True)
print(f"number of rows after dropping duplicates: {len(bills_topics)}")  #5373

number of rows before dropping duplicates: 153220
number of rows after dropping duplicates: 5373


In [5]:
#split topics into database and topics
topics_split = bills_topics["ncsl_topics"].str.split("__", expand = True).rename(columns={1: "database", 3: "ncsl_topic"})
print(topics_split[0].unique())                         #unique : ncsl_database & None
print(topics_split[2].unique())                         #unique : ncsl_topic & None
topics_split.drop(columns=[0,2], inplace=True)          #remove redunant cols
bills_topics.drop(columns=["ncsl_topics"], inplace=True)#remove ncsl_topic col from bills_topics_all

#merge the topics split with bills_toppics
bills_topics = bills_topics.merge(topics_split, left_index=True, right_index=True)

['ncsl_database' None]
['ncsl_topic' None]


In [7]:
bills_topics["ncsl_metatopics"].unique()

array(['military_and_veterans_affairs',
       'environment_and_natural_resources', 'elections_and_campaigns',
       'human_services', 'education', 'transportation',
       'labor_and_employment', 'energy', 'financial_services', 'fiscal',
       'quad_caucus', 'civil_and_criminal_justice', 'ethics', 'health',
       'technology_and_communication', 'immigration',
       'redistricting_and_census'], dtype=object)

In [29]:
bills_topics

,ncsl_metatopics,database,ncsl_topic
0,military_and_veterans_affairs,military_and_veterans_state_legislation_database,education
1,environment_and_natural_resources,None,None
2,elections_and_campaigns,state_elections_legislation_database,election_crimes
3,human_services,child_support_and_family_law_legislation_database,enforcement
4,education,education_legislation_bill_tracking,career_and_technical_education
...,...,...,...
5368,technology_and_communication,injury_prevention_legislation_database_opioid_...,prescription_drug_monitoring_program
5369,military_and_veterans_affairs,health_innovations_state_law_archive_database_...,medicaid_expansion_and_waivers
5370,military_and_veterans_affairs,child_welfare_enacted_legislation,courts_and_legal_representation
5371,technology_and_communication,covid_19_archive_database,legislative_operations


In [18]:
bills_topics_envi = bills_topics[bills_topics["ncsl_metatopics"] == "environment_and_natural_resources"]["ncsl_topic"].drop_duplicates().to_list()

In [19]:
bills_topics_envi

[None,
 'food_safety',
 'water_resources/planning',
 'body_art_tattoo',
 'eh_management',
 'indoor_air_quality',
 'water_waste_water',
 'drinking_water',
 'pesticides',
 'toxics_&_chemicals',
 'waste_and_recycling',
 'indoor_air_quality_–_radon',
 "children's_environmental_health",
 'waste_and_recycling_product_stewardship',
 'waste_and_recycling_composting',
 'transportation',
 'transportation_alt_fuel/hybrid',
 'energy_security_and_critical_infrastructure',
 '9_1_1_administration,_plans,_boards_&_commissions',
 '9_1_1_funding_and_appropriations',
 'enhanced_9_1_1',
 'waste_and_recycling_beverage_containers',
 'asthma',
 'energy_efficiency',
 'energy_efficiency_building_codes_and_standards',
 'water_conservation/efficiency',
 'nuclear_/_radioactive_waste',
 'nuclear_energy_facilities',
 'waste_nuclear/radioactive',
 'tax_credits_&_incentives_evaluation,_reporting_&_rqmts',
 'tax_credits_&_incentives_small_&_minority_businesses',
 'tax_credits_&_incentives_sunset_provisions',
 'tax_cre

In [ ]:
# identify unique ncsl metatopics
bills_meta = bills["ncsl_metatopics"].dropna()
bills_meta_unique = bills_meta.str.split("; ").explode().unique()
bills_meta_unique = [x.replace("ncsl_metatopic__", "") for x in bills_meta_unique]
print(len(bills_meta_unique)) #17
bills_meta_unique

17


['military_and_veterans_affairs',
 'environment_and_natural_resources',
 'elections_and_campaigns',
 'human_services',
 'education',
 'transportation',
 'labor_and_employment',
 'energy',
 'financial_services',
 'fiscal',
 'quad_caucus',
 'civil_and_criminal_justice',
 'ethics',
 'health',
 'technology_and_communication',
 'immigration',
 'redistricting_and_census']

In [ ]:
#bills_meta = [x.split("; ") for x in bills_meta] # split strings where there are multiple topics
#bills_meta = list(itertools.chain.from_iterable(bills_meta)) # flatten list

In [ ]:
bills_topics["ncsl_topic"].str.contains("recycling|waste|plastic")

0       False
1        None
2       False
3       False
4       False
        ...  
5368    False
5369    False
5370    False
5371    False
5372    False
Name: ncsl_topic, Length: 5373, dtype: object

## data: POSITIONS

In [ ]:
#OVERVIEW:
#col labels
print(positions.columns)
#length
print(len(positions)) #13619409
#length positions per state
positions.value_counts("state")
#positions

Index(['bill_version', 'client_name', 'lobbyist_rep_name',
       'lobbyist_firm_name', 'position', 'start_date', 'end_date', 'year',
       'record_type', 'session', 'description', 'committee', 'state',
       'position_numeric', 'docket_number', 'docket_prefix',
       'legiscan_bill_id', 'state_unified_bill_id', 'state_client_id',
       'ncsl_metatopics', 'ncsl_topics'],
      dtype='object')
13619409


state
AZ    5736266
IL    3176041
CO    1704725
IA     789782
TX     647280
MA     475652
NE     307651
MT     154337
MD      99080
WI      98712
NJ      70361
RI      67721
KS      65721
SD      65505
MO      64849
FL      53305
OH      42421
Name: count, dtype: int64

In [11]:
positions[positions.duplicated("state_unified_bill_id", keep = False)].sort_values(by = ["state_unified_bill_id", "client_name"])

,bill_version,client_name,lobbyist_rep_name,lobbyist_firm_name,position,start_date,end_date,year,record_type,session,...,committee,state,position_numeric,docket_number,docket_prefix,legiscan_bill_id,state_unified_bill_id,state_client_id,ncsl_metatopics,ncsl_topics
13282944,20765,Administrative Office of fhe Courts,Jerry Landau,None,AGAINST,2006-01-18 07:07:42+00:00,2006-01-18 07:07:42+00:00,2006,testimony,2006 - Forty-seventh Legislature - Second Regu...,...,None,AZ,-1.0,None,None,None,AZ_HB_0000002001_2006,AZ_28,None,None
13283128,20765,Administrative Office of fhe Courts,Jerry Landau,None,AGAINST,2006-01-18 07:07:42+00:00,2006-01-18 07:07:42+00:00,2006,testimony,2006 - Forty-seventh Legislature - Second Regu...,...,None,AZ,-1.0,None,None,None,AZ_HB_0000002001_2006,AZ_28,None,None
13295140,20765,Administrative Office of fhe Courts,Jerry Landau,None,AGAINST,2006-01-18 07:07:42+00:00,2006-01-18 07:07:42+00:00,2006,testimony,2006 - Forty-seventh Legislature - Second Regu...,...,None,AZ,-1.0,None,None,None,AZ_HB_0000002001_2006,AZ_28,None,None
13295839,20765,Administrative Office of fhe Courts,Jerry Landau,None,AGAINST,2006-01-18 07:07:42+00:00,2006-01-18 07:07:42+00:00,2006,testimony,2006 - Forty-seventh Legislature - Second Regu...,...,None,AZ,-1.0,None,None,None,AZ_HB_0000002001_2006,AZ_28,None,None
13297214,20765,Administrative Office of fhe Courts,Jerry Landau,None,AGAINST,2006-01-18 07:07:42+00:00,2006-01-18 07:07:42+00:00,2006,testimony,2006 - Forty-seventh Legislature - Second Regu...,...,None,AZ,-1.0,None,None,None,AZ_HB_0000002001_2006,AZ_28,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664161,3635.0,Wisconsin Technical College District Boards As...,None,None,AGAINST,2006-02-22 00:00:00+00:00,NaT,2006,lobbying,2005REG,...,None,WI,-1.0,None,None,None,WI_SR_0000000063_2005,WI_163,None,None
1663869,3635.0,Wisconsin Tourism Federation,None,None,OTHER,2006-04-04 00:00:00+00:00,NaT,2006,lobbying,2005REG,...,None,WI,0.0,None,None,None,WI_SR_0000000063_2005,WI_705,None,None
1662696,3635.0,Wisconsin Transportation Builders Association,None,None,NONE,2006-04-12 00:00:00+00:00,NaT,2006,lobbying,2005REG,...,None,WI,0.0,None,None,None,WI_SR_0000000063_2005,WI_132,None,None
1673606,5054.0,American Diabetes Association,None,None,FOR,2007-10-29 00:00:00+00:00,NaT,2007,lobbying,2007REG,...,None,WI,1.0,None,None,None,WI_SR_0000000065_2007,WI_790,None,None
